# Memory

In [30]:
import { ChatMessageHistory } from 'langchain/stores/message/in_memory'
import { HumanMessage, AIMessage } from '@langchain/core/messages'

const history = new ChatMessageHistory()

await history.addMessage(new HumanMessage('Hello!'))
await history.addMessage(new AIMessage('Hi!'))

const messages = await history.getMessages()
console.log(messages)

[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: { content: "Hello!", additional_kwargs: {}, response_metadata: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "Hello!",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: { content: "Hi!", additional_kwargs: {}, response_metadata: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "Hi!",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  }
]


In [31]:
// import env config

import { load } from "dotenv";
const env = await load();

const process = {
    env
}

In [32]:
import { ChatPromptTemplate, MessagesPlaceholder } from '@langchain/core/prompts'

import { ChatOpenAI } from '@langchain/openai'

const chatModel = new ChatOpenAI()

const prompt = ChatPromptTemplate.fromMessages([
    ["system", `You are a helpful assistant. Answer all questions to the best of your ability.
        You are talkative and provides lots of specific details from its context. 
        If the you does not know the answer to a question, it truthfully says you do not know.`],
    new MessagesPlaceholder('history_message')
])

const chain = prompt.pipe(chatModel)

In [33]:
import { ChatMessageHistory } from 'langchain/stores/message/in_memory'
import { HumanMessage } from '@langchain/core/messages'

const history = new ChatMessageHistory()
await history.addMessage(new HumanMessage('Hello! My name is Bob.'));

const res = await chain.invoke({
    history_message: await history.getMessages()
})

console.log(res)

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "Hello Bob! How can I assist you today?",
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "Hello Bob! How can I assist you today?",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined },
  response_metadata: {
    tokenUsage: { completionTokens: 10, promptTokens: 72, totalTokens: 82 },
    finish_reason: "stop"
  }
}


In [34]:
await history.addMessage(res)
await history.addMessage(new HumanMessage('What is my name?'))

const res2 = await chain.invoke({
    history_message: await history.getMessages()
})

console.log(res2)

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "Your name is Bob.",
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "Your name is Bob.",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined },
  response_metadata: {
    tokenUsage: { completionTokens: 5, promptTokens: 95, totalTokens: 100 },
    finish_reason: "stop"
  }
}


## Manully manage memory

In [67]:
import { ChatOpenAI } from '@langchain/openai'
import { ChatPromptTemplate, MessagesPlaceholder } from '@langchain/core/prompts'
import { ChatMessageHistory } from 'langchain/stores/message/in_memory'
import { RunnableWithMessageHistory } from '@langchain/core/runnables'


const chatModel = new ChatOpenAI()
const prompt = ChatPromptTemplate.fromMessages([
    ["system", `You are a helpful assistant. Answer all questions to the best of your ability.
        You are talkative and provides lots of specific details from its context. 
        If the you does not know the answer to a question, it truthfully says you do not know.`],
    new MessagesPlaceholder('history_message'),
    ['human', '{input}']
])

const history = new ChatMessageHistory()
const chain = prompt.pipe(chatModel)

const chainWithHistory = new RunnableWithMessageHistory({
    runnable: chain,
    getMessageHistory: _sessionId => history,
    inputMessageKey: 'input',
    historyMessagesKey: 'history_message' // spelling issue 's' missing
})


In [69]:
const res1 = await chainWithHistory.invoke({
    input: 'Hello! My name is Bob.',
},{
    configurable: { sessionId: "none" }
})

console.log(res1)

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "Hello, Bob! How can I assist you today?",
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "Hello, Bob! How can I assist you today?",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined },
  response_metadata: {
    tokenUsage: { completionTokens: 11, promptTokens: 104, totalTokens: 115 },
    finish_reason: "stop"
  }
}


In [70]:
const res2 = await chainWithHistory.invoke({
    input: 'What is my name?',
},{
    configurable: { sessionId: "none" }
})

console.log(res2)

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "Your name is Bob.",
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "Your name is Bob.",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined },
  response_metadata: {
    tokenUsage: { completionTokens: 5, promptTokens: 128, totalTokens: 133 },
    finish_reason: "stop"
  }
}


In [71]:
await history.getMessages()

[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "hi, my name is Kai",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "hi, my name is Kai",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "Hello, Kai! It's nice to meet you. How can I assist you today?",
      additional_kwargs: { function_call: undefined, tool_calls: undefined },
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "Hello, Kai! It's nice to meet you. How can I assist you today?",
    name: undefined,
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {
      tokenUsage: { completionTokens: 18, promptTokens: 71, totalTokens: 89 },
      finish_reason: "stop"
    }
  },
  HumanMessage {
    lc_serializable: true,
   

## Auto Chat history abstraction

In [72]:
import { RunnableWithMessageHistory } from "@langchain/core/runnables";
import { ChatPromptTemplate, MessagesPlaceholder } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";
import { ChatMessageHistory } from "langchain/stores/message/in_memory";
import { RunnableSequence } from "@langchain/core/runnables";
import { RunnablePassthrough } from "@langchain/core/runnables";
import { StringOutputParser } from "@langchain/core/output_parsers";
import { getBufferString } from "@langchain/core/messages";

In [73]:
const summaryModel = new ChatOpenAI();
const summaryPrompt = ChatPromptTemplate.fromTemplate(`
Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:
`); 

const summaryChain = RunnableSequence.from([
    summaryPrompt,
    summaryModel,
    new StringOutputParser(),
])

In [77]:
const newSummary = await summaryChain.invoke({
    summary: '',
    new_lines: "I'm 18"
})

In [78]:
await summaryChain.invoke({
    summary: newSummary,
    new_lines: "I'm male"
})

"The person identifies as an 18-year-old male."

In [79]:
const chatModel = new ChatOpenAI();
const chatPrompt = ChatPromptTemplate.fromMessages([
    ["system", `You are a helpful assistant. Answer all questions to the best of your ability.

    Here is the chat history summary:
    {history_summary}
    `],
    ["human","{input}"]
]);
let summary = ""
const history = new ChatMessageHistory();

const chatChain = RunnableSequence.from([
    {
        input: new RunnablePassthrough({
             func: (input) => history.addUserMessage(input)
        })
    },
    RunnablePassthrough.assign({
        history_summary: () => summary
    }),
    chatPrompt,
    chatModel,
    new StringOutputParser(),
    new RunnablePassthrough({
        func: async (input) => {
            history.addAIChatMessage(input)
            const messages = await history.getMessages()
            const new_lines = getBufferString(messages)
            const newSummary = await summaryChain.invoke({
                summary,
                new_lines
            })
            console.log(summary, input, messages, new_lines, newSummary)
            history.clear()
            summary = newSummary      
        }
    })
])

In [80]:
await chatChain.invoke("Hello! My name is Bob.")

"Hello Bob! How can I assist you today?"

In [81]:
console.log(summary)

Bob introduces himself and the AI responds by greeting him and offering assistance.


In [82]:
await chatChain.invoke("I'm tired now")

"I'm sorry to hear that you're tired. Is there anything I can do to help?"

In [83]:
console.log(summary)

Bob mentions that he is tired, to which the AI expresses sympathy and offers assistance.


In [85]:
await chatChain.invoke("i hope the world without wars")

Bob expresses his desire for a world without wars, to which the AI acknowledges the noble goal and highlights the importance of individuals promoting peace in their own lives and communities. The AI suggests spreading empathy, understanding, and actively working towards peacefully resolving conflicts as ways to contribute to a more peaceful world. I understand your desire for a world without wars. It's a noble goal that many people share. While achieving a completely war-free world may be challenging, it is important for individuals to promote peace in their own lives and communities. Spreading empathy and understanding, and actively working towards peacefully resolving conflicts can all contribute to a more peaceful world. It starts with small actions, and by collectively working towards peace, we can make a difference. [
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "i hope the world without wars",
      additional_kwargs: {},
      response_metadata: {}

"I understand your desire for a world without wars. It's a noble goal that many people share. While a"... 382 more characters